<a href="https://colab.research.google.com/github/meghana-0211/crop-prediction-model/blob/main/iiot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Load the dataset
df = pd.read_csv('Crop_recommendation.csv')

# Prepare features and target variable
df['target'] = df.label.astype('category').cat.codes
y = df['target']
# Only consider temperature, humidity, and rainfall as features
X = df[['temperature', 'humidity', 'rainfall']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# K-Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

# Finding optimal k for KNN
k_range = range(1, 11)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    scores.append(knn.score(X_test_scaled, y_test))

# Support Vector Machine with various kernels
svc_linear = SVC(kernel='linear').fit(X_train_scaled, y_train)
svc_poly_rbf = SVC(kernel='rbf').fit(X_train_scaled, y_train)
svc_poly = SVC(kernel='poly').fit(X_train_scaled, y_train)

# Hyperparameter tuning for SVM with linear kernel
parameters = {'C': np.logspace(-3, 2, 6).tolist(), 'gamma': np.logspace(-3, 2, 6).tolist()}
model = GridSearchCV(estimator=SVC(kernel="linear"), param_grid=parameters, n_jobs=-1, cv=4)
model.fit(X_train, y_train)

# Random Forest Classifier
clf = RandomForestClassifier(max_depth=4, n_estimators=100, random_state=42).fit(X_train, y_train)

# Gradient Boosting Classifier
grad = GradientBoostingClassifier().fit(X_train, y_train)

# Print the accuracy of the Gradient Boosting model
print('Gradient Boosting accuracy: {:.2f}%'.format(grad.score(X_test, y_test) * 100))


Gradient Boosting accuracy: 92.55%


In [ ]:
def predict_crop(temp, humidity, rainfall):
    # Prepare input features
    input_data = pd.DataFrame([[temp, humidity, rainfall]],
                              columns=['temperature', 'humidity', 'rainfall'])

    # Scale the input data
    input_scaled = scaler.transform(input_data)

    # Predict crop label using the Gradient Boosting model
    predicted_label = grad.predict(input_scaled)[0]

    # Map the label number to crop name
    crop_name = df[df['target'] == predicted_label]['label'].iloc[0]

    return crop_name

In [ ]:
print("Predicted Crop:", predict_crop(25, 40, 240))

Predicted Crop: muskmelon


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
